# Persistence

## Simplex def

In [1]:
class Simplex:
    def __init__(self, val, dim, vert):
        self.val = val
        self.dim = dim
        self.vert = vert
        
    def __eq__(self, simplex2):
        return self.vetices.__eq__(simplex2.verticles)
    

    def toString(self):
        return "val = {}; dim = {}; vert = {}".format(self.val, self.dim, "/".join(self.vert))

## Load the data

In [2]:
simplex_list = []

with open("filtrations/test_filtration2.txt", "r") as test_filtration:
    for line in test_filtration.readlines():        
        inputs = line.strip().split(" ")
        f, dim = float(inputs[0]), int(inputs[1])
        vertices = sorted(list(map(int, inputs[2:]))) # vertices are sorted so we can compare between list of vertices
        simplex_list.append(Simplex(f, dim, vertices))
        
simplex_list = sorted(simplex_list, key=lambda x:(x.val, x.dim)) # sort according to val and then to dim (to have a simplicial complex)


dic_simplex = {} # maps a simplex (as a sorted tuple of vertices) to its indentifier 
for i, s in enumerate(simplex_list):
    dic_simplex[tuple(s.vert)] = i
        

FileNotFoundError: [Errno 2] No such file or directory: 'filtrations/test_filtration2.txt'

## Boundary matrix

In [ ]:
def compute_boundary(simplex, dic_simplex):
    """return the identifiers of the boundaries of the simplex"""
    boundaries_id = []
    for i in range(simplex.dim + 1):
        new_vert = list(simplex.vert)
        del new_vert[i]
        if len(new_vert) > 0:
            boundaries_id.append(dic_simplex[tuple(sorted(new_vert))]) # new_vert must be sorted because we compare tuple (it's actually already sorted but we sort it for code clarity)
    return boundaries_id
        

In [ ]:
# We define low(i) = the maximal non-zero element for the i-th simplex.

# dic_low[k] = list containing all the simplices i whose low(i) is in the k-th row, i.e. corresponds to the k-th simplex
# dic_boundaries[i] = list containing all the non-zero boundary simplices from the i-th simplex
dic_low, dic_boundaries = {},{} 
for i in range(len(simplex_list)):
    dic_boundaries[i] = [] # initialization
    dic_low[i] = []

for simplex in simplex_list:
    
    simplex_vert = tuple(sorted(simplex.vert))
    simplex_coord = dic_simplex[simplex_vert] #identifier of the simplex
    boundaries_id = compute_boundary(simplex, dic_simplex)
    if len(boundaries_id) > 0:
        dic_boundaries[simplex_coord] = boundaries_id
        low_i = max(boundaries_id)
        # Check if they're already another simplex with the same low(i)
        dic_low[low_i] = dic_low[low_i] + [simplex_coord] if (low_i in dic_low.keys()) else [simplex_coord] 
    for boundary_id in boundaries_id:
         #identifier of the boundary simplex
        dic_coord[(simplex_coord, boundary_id)] = 1
        


In [ ]:
dic_boundaries

## Reduction

In [ ]:
def sum_boundaries(boundaries1, boundaries2):
    """sum the list of boundaries of 2 simplex in Z/2Z
    Complexity could be improved"""
    res = []
    for b in boundaries1:
        if b not in boundaries2:
            res.append(b)
    for b in boundaries2:
        if b not in res and b not in boundaries1:
            res.append(b)
    return res
        

def gaussian_elim(dic_low, dic_boundaries):
    pivots = []
    for i in reversed(range(0, len(dic_boundaries.keys()))):
        if i in dic_low.keys():
            columns = sorted(dic_low[i])
            if len(columns) > 0:  # if there are columns to kill
                pivot_column = columns[0]
                pivots.append(pivot_column)
                for column in columns[1:]:
                    # sum in Z/2Z
                    dic_boundaries[column] = sum_boundaries(dic_boundaries[column], dic_boundaries[pivot_column])
                    if len(dic_boundaries[column]) > 0: 
                        # update the pivot
                        new_pivot = max(dic_boundaries[column])
                        if new_pivot in dic_low.keys():
                            dic_low[new_pivot].append(column)
                        else:
                            dic_low[new_pivot] = [column]
                            
    return pivots, dic_boundaries
            
pivots, dic_boundaries = gaussian_elim(dic_low, dic_boundaries)

In [ ]:
dic_boundaries

## Bar code

In [ ]:
def bar_codes(simplex_list, dic_boundaries):
    threshold = 0.05

    bar_codes = {}
    for i, s in enumerate(simplex_list):
        if len(dic_boundaries[i]) == 0: #create a cycle
            bar_codes[i] = None
        else: #kill a cycle (create a boundary)
            k = max(dic_boundaries[i]) # pivot
            if k in bar_codes.keys():
                bar_codes[k] = i

    for s1_id in bar_codes.keys():
        dim = simplex_list[s1_id].dim
        s2_id = bar_codes[s1_id]
        s1_val = simplex_list[s1_id].val
        if s2_id: # the cycle as been killed
            s2_val = simplex_list[s2_id].val
            if s2_val - s1_val > threshold:
                print(dim, s1_val, s2_val)
        else: # the cycle has never been killed
            s2_val = "inf"
            print(dim, s1_val, s2_val)
        
bar_codes(simplex_list, dic_boundaries)

In [85]:
import itertools as itertools

def top_simplices_from_figure(list_vertices):
    # Useful function to list the top-dimensional subsimplices from the Moebius band, the torus and the Klein bottle.
    """
    Input: 2D-list from the vertices, row by row
    Output: set from the top-dimensional simplices (here the 2D simplices), i.e. set of list of vertices
    """
    top_simplices = [] 
    for i in range(len(list_vertices) - 1):
        for j in range(len(list_vertices[i]) - 1):
            top_simplices.append((list_vertices[i][j],list_vertices[i][j+1],list_vertices[i+1][j]))
            top_simplices.append((list_vertices[i][j+1],list_vertices[i+1][j],list_vertices[i+1][j+1]))
    return set(top_simplices)

def subsimplices_from_simplex(simplex_vert):
    """
    Input: tuple from the vertices of the simplex
    Output: set from the subsimplices, i.e. set of tuple of vertices. Includes the input simplex.
    """
    return set(itertools.chain(*[itertools.combinations(simplex_vert, dim) for dim in range(1,len(simplex_vert) + 1)]))


def from_simplices_to_filtration(simplices_set):
    """
    Input: set from the simplices, i.e. set of tuples of vertices
    Output: filtration = list from the simplices in crescent order for the dimension
    (according to the syntax 
        Time of appearance in the filtration | Dimension | List from the vertices
    )
    In our filtration : 
        Time of appearance(simplex K) = dim(simplex K)
    """
    all_simplices = simplices_set.copy()
    for simplex_vert in simplices_set:
        new_simplices = subsimplices_from_simplex(simplex_vert)
        for new_simplex in new_simplices:
            all_simplices.add(new_simplex)
    all_simplices = sorted(list(all_simplices), key = lambda x:(len(x),x[0]))
    
    filtration = []
    for ord_simplex in all_simplices:
        new_filtration = str(len(ord_simplex)-1) + str(' ') + str(len(ord_simplex)-1) + str(' ') + "".join(str(ord_simplex[i]) + str(' ') for i in range(len(ord_simplex)))
        filtration.append(new_filtration)
    return filtration
    
def compute_filtrations(d):
    # Compute all the required filtrations
    """
    Input: wanted dimension for the d-ball and for the d-sphere
    Output: list with all the required filtrations, according to the syntax given in "from simplices to filtration"
    """
    all_filtrations = []
    
    # For the d-ball
    all_filtrations.append(from_simplices_to_filtration(subsimplices_from_simplex(top_simplex_d_ball(d))))
    
    # For the d-sphere    
    all_filtrations.append(from_simplices_to_filtration(subsimplices_from_simplex(top_simplex_d_ball(d)))[:-1]) # We remove the (d+1) simplex

    # For the Moebius band
    all_filtrations.append(from_simplices_to_filtration(top_simplices_from_figure(triangulation_Moebius_band())))
    
    # For the torus
    all_filtrations.append(from_simplices_to_filtration(top_simplices_from_figure(triangulation_torus())))
    
    # For the Klein bottle
    all_filtrations.append(from_simplices_to_filtration(top_simplices_from_figure(triangulation_Klein_bottle())))

    # For the projective plane
    all_filtrations.append(from_simplices_to_filtration(top_simplices_projective_plane()))
    
    return all_filtrations
    
def triangulation_Moebius_band():
    """
    Output: 2D-list from the vertices, row by row, corresponding to a triangulation from the Moebius band
    """
    return [['A','B','C','D'],['D','E','F','A']]

def triangulation_torus():
    """
    Output: 2D-list from the vertices, row by row, corresponding to a triangulation from the torus
    """
    return [['A','B','C','A'],['D','E','F','D'],['G','H','I','G'],['A','B','C','A']]

def triangulation_Klein_bottle():
    """
    Output: 2D-list from the vertices, row by row, corresponding to a triangulation from the Klein bottle
    """
    return [['A','B','C','A'],['D','E','F','G'],['G','H','I','D'],['A','B','C','A']]

def top_simplices_projective_plane():
    """
    Output: set from the top-dimensional simplices (here the 2D simplices from a triangulation from the 2D projective plane), i.e. set of list of vertices
    """
    return set([('A','B','D'),('B','C','D'),('C','D','E'),('C','E','A'),('A','E','B'),('B','E','F'),('B','C','F'),('A','F','C'),('A','F','D'),('D','E','F')])

def top_simplex_d_ball(d):
    """
    Input: the dimension d
    Output: tuple of the (d+1) vertices from the top-dimensional simplex of the d-ball
    """
    assert d < 11 # We could have gone further, one can easily modify it by extending list_max_vertices
    list_vertices = ['A','B','C','D','E','F','G','H','I','J','K']
    return tuple(list_vertices[:d+1])
